In [1]:
import pandas as pd
import os

data = pd.read_csv(os.getcwd()+'/data/BNP_data/train.csv')

# Fill NA values accordingly
from sklearn.base import TransformerMixin
import numpy as np

class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
data = DataFrameImputer().fit_transform(data)

# Convert string values to numeric
from sklearn import preprocessing

criteria = data.dtypes == object
object_columns = data[criteria.index[criteria]].columns

le = preprocessing.LabelEncoder()
for col in object_columns:
    data[col] = le.fit_transform(data[col])

# Cross-validation
from sklearn import cross_validation
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(data.drop('target', axis=1), data['target'], \
    test_size=0.33, random_state=5)

In [5]:
cat_cols = {}
for col_name in data.columns:
    unique_count = len(data[col_name].unique())
    if unique_count < 20:
        cat_cols[col_name] = unique_count

cat_cols

{'target': 2,
 'v107': 7,
 'v110': 3,
 'v129': 10,
 'v24': 5,
 'v3': 3,
 'v30': 7,
 'v31': 3,
 'v38': 12,
 'v47': 10,
 'v52': 12,
 'v62': 8,
 'v66': 3,
 'v71': 9,
 'v72': 13,
 'v74': 3,
 'v75': 4,
 'v79': 18,
 'v91': 7}

In [9]:
test_group = data[['target', 'v3']].groupby(data['target'])

In [17]:
len(test_group.groups[1])/float(data.shape[0])

0.7611987298921458

In [43]:
from scipy.stats.stats import normaltest
import numpy as np

close_to_normal = {}
for name in data.drop('target', axis=1).columns:
    if normaltest(data[name]).pvalue > 0.05:
        close_to_normal[name] = normaltest(data[name])
        
close_to_normal

{}

In [38]:
data.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,2,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,21,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,1.469550,7.023803,2,4.205991,9.191265,2.412790,2.452166,2.301630,...,6.749117,2.739239,0.598896,6,1.614802,2.963620,1.957825,0,1.560138,1.589403
2,5,1,0.943877,5.310079,2,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,5,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,2,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,64,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,1.469550,7.023803,2,4.205991,8.670867,2.412790,2.452166,0.386032,...,6.749117,2.739239,0.139864,89,1.614802,2.963620,1.798436,0,1.560138,1.589403
